# Autoencoder

## 调包

In [1]:
import os
import numpy as np
from tqdm import tqdm
import torch
import torch.jit
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import tensorflow as tf
#from openfe import openfe, transform
#import graphviz

from datetime import timedelta
import warnings; 
warnings.filterwarnings("ignore")

import pandas as pd 
from pandas.tseries.offsets import Day, MonthEnd, MonthBegin # 时间处理
import numpy as np
import pyarrow.feather as feather
import matplotlib.pyplot as plt 

#import optuna
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, KBinsDiscretizer, LabelEncoder, OrdinalEncoder


from torch.utils.data import DataLoader
from torch.utils.data import Dataset

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)


## GPU

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


## AutoEncoder

## Model

In [92]:
class Factor_data(Dataset):
    def __init__(self, train_x, train_y): #默认输入的时候就已经是tensor
        self.len = len(train_x)
        self.x_data = train_x
        self.y_data = train_y
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    def __len__(self):
        return self.len

In [93]:
class Factor_models(object):

    def __init__(self,data,train_period,freq='m'):
        '''
        参数初始化
        '''
    
        self.data = data
        self.train_period = train_period
        self.freq = freq # 训练集expanding的频率，是按月还是按年还是其他
        
    def predict_ret(self):
        dates = self.data.index.unique()
        dates = dates.sort_values()
        print(dates)
        test_dates= dates[self.train_period:len(dates)]
        print(test_dates)
        
        # 创建一个train_end_list，训练集每月expanding。
        preddf = pd.DataFrame() # 存储不同模型预测出来的y值，即存储样本外预测收益率的值
        
        
        index = 0  # 计数用

        for end_date in tqdm(test_dates,desc='Spilt and Train'): # 通过逐月改变训练集end_date的方法，切割样本
            
            temp = self.data[(self.data.index < end_date)]
            valid_temp = self.data[self.data.index == end_date]
            
            # 训练集
            train_y = temp.ret
            train_x = temp.drop(['ret'], axis=1)
    

            # 测试集
            #测试集为训练集的次月，只预测下个月的月度收益率
            valid_y = valid_temp.ret
            valid_x = valid_temp.drop(['ret'], axis=1)

            #分割给optuna用的数据集
            X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=0)

            
            # 数据标准化
            st = StandardScaler()
            sv = StandardScaler()
            s1 = StandardScaler()
            s2 = StandardScaler()
            
            train_x = st.fit_transform(train_x)
            valid_x = sv.fit_transform(valid_x)
            X_train = s1.fit_transform(X_train)
            X_test = s2.fit_transform(X_test)            

            train_y = train_y.values.reshape(-1, 1)
            valid_y = valid_y.values.reshape(-1,1)
            y_train = y_train.values.reshape(-1,1)
            y_test = y_test.values.reshape(-1,1)
                                 
            # 建模预测收益率
            ## 先创建一个临时的temp_preddf,用来存储当前月份的验证集下的real y和不同模型的预测y
            temp_preddf = pd.DataFrame() # 创建当前训练集下训练出的predict y和real y
            temp_preddf['real_y'] = valid_y[:,0]# real_y就是验证集valid_y的第一列。因为valid_y是真实收益率数据在vailid_date上的切割
            
            ## NN3,未调参
            ### 三层的神经网络       
            train_x = torch.tensor(train_x,dtype = torch.float)
            train_y = torch.tensor(train_y,dtype = torch.float)

            valid_x = torch.tensor(valid_x,dtype = torch.float)
            valid_y = torch.tensor(valid_y,dtype = torch.float)
            
            X_train = torch.tensor(X_train,dtype = torch.float)
            X_test = torch.tensor(X_test,dtype = torch.float)
            y_train = torch.tensor(y_train,dtype = torch.float)
            y_test = torch.tensor(y_test,dtype = torch.float)

            train_x =  train_x.to(device)
            train_y = train_y.to(device)

            valid_x = valid_x.to(device)
            valid_y = valid_y.to(device) 

            X_train = X_train.to(device)
            y_train = y_train.to(device)
            X_test = X_test.to(device)
            y_test = y_test.to(device) 

            
            
            batch_size = 40000
            train_data = Factor_data(train_x,train_y) 
            train_loader = DataLoader(dataset = train_data,
                                     batch_size = batch_size,
                                     shuffle = False) #不打乱数据集
            
        
            
            # Create model
            model = FactorAE().to(device)

            #weight decay: 对所有weight参数进行L2正则化
            weight_list,bias_list = [],[]
            for name,p in model.named_parameters():
                if 'bias' in name:
                    bias_list += [p]
                else:
                    weight_list += [p]

            optimizer = optim.SGD([{'params': weight_list, 'weight_decay':1e-5},{'params': bias_list, 'weight_decay':0}],lr = 1e-3,momentum = 0.9)

            #Xavier Initialization
            for n in model.modules():  
                if isinstance(n,nn.Linear): #线性全连接层初始化 
                    n.weight = nn.init.xavier_normal_(n.weight, gain=1.)   
        
            #training        
            criterion = nn.MSELoss()
            epoch_num = 10
            loss_list = []
            for epoch in range(epoch_num):
                train_loss = 0.0

                for data,y_return in train_loader:
                    output = model(data,y_return)
                    loss = criterion(output,y_return)
                    optimizer.zero_grad()
                    train_loss += loss
                    loss.backward()
                    optimizer.step()

            predict_y = model(train_x,train_y)
            predict_y = predict_y.detach().cpu().numpy()
            temp_preddf['AEncoder_y'] = predict_y
              
            ## 将temp_preddf并入preddf
            preddf = preddf.append(temp_preddf) # 将当前valid_date下得到的predict_y和real_y一起并入preddf中
            self._preddf = preddf

        torch.save(net, '/home/mw/project/NN/AEncoder_tuning.pt')
        return preddf # 最后我们只返回preddf，也就是所有期的predict y和real y
    
    def cal_oos(self):
        # 计算out-of-sample R2 根据代码开头的公式
        try:
            preddf = self._preddf # 如果self已经有self._preddf，即self.predict_ret()已经运行过了，已经预测过收益率了，则无需再次运行。
        except:
            preddf = self.predict_ret() # 如果之前没有运行过self.predict_ret()，则需要运行。
        denominator = (preddf['real_y'] ** 2).sum() # 分母是真实收益率的平方和
        numerator = preddf.apply(lambda x: preddf['real_y'] - x).iloc[:,1:] # 分子是real_y - predict_y的平方和
        numerator = (numerator ** 2).sum()
        
        roos = 1 - numerator / denominator # 再用 1 减去分子/分母
        roos.index = roos.index.str.rstrip('_y') # 之前的index都是模型_y，比如"OLS_y"，不美观，删除_y。
        fig,ax = plt.subplots(figsize = (16,12)) # 画图，将不同模型的Roos画出来。
        plt.title('Out-of-sample predicting R2', fontsize = 20)
        ax.bar(x = roos.index, height = roos)
        plt.show()
        return roos # 返回样本外Roos，这个Roos是不同模型对应的样本外R2

## Main

In [11]:
#导入数据
with open('/home/mw/input/stock3636/chars60_rank_imputed.feather', 'rb') as f:
    data = feather.read_feather(f)
data['date'] = data['date'].astype('datetime64')
data_3d = data.copy()
data.set_index('date', inplace=True)

#data_3d.set_index(["date","permno"], inplace=True)
# 缺失值处理
## 查看缺失值--没有缺失值
print('Missing data: {} items\n'.format(len(data[data.isna().any(1)])), data[data.isna().any(1)]) # 看一下缺失值是哪些行

s = (data.dtypes == 'int64')
object_cols = list(s[s].index)# 移除含有类别变量的列
# 移除数据集含有类别变量的列（因为我们已经获得了含有独热编码的列）
data = data.drop(object_cols, axis=1)
data_3d = data_3d.drop(['gvkey','sic','ffi49'],axis=1)

Missing data: 0 items
 Empty DataFrame
Columns: [gvkey, permno, sic, ret, exchcd, shrcd, ffi49, lag_me, rank_acc, rank_cash, rank_maxret, rank_lgr, rank_roe, rank_sgr, rank_bm, rank_noa, rank_rdm, rank_chtx, rank_chpm, rank_cashdebt, rank_mom12m, rank_turn, rank_rvar_capm, rank_ato, rank_pscore, rank_chcsho, rank_beta, rank_dolvol, rank_alm, rank_std_dolvol, rank_rvar_mean, rank_rd_sale, rank_abr, rank_sp, rank_mom60m, rank_pctacc, rank_ill, rank_lev, rank_rna, rank_mom6m, rank_seas1a, rank_nincr, rank_mom1m, rank_zerotrade, rank_cfp, rank_pm, rank_me, rank_ni, rank_mom36m, rank_rvar_ff3, rank_gma, rank_roa, rank_rsup, rank_baspread, rank_sue, rank_grltnoa, rank_std_turn, rank_depr, rank_cinvest, rank_op, rank_agr, rank_ep, log_me]
Index: []

[0 rows x 63 columns]


In [12]:
data

,ret,exchcd,shrcd,lag_me,rank_acc,rank_cash,rank_maxret,rank_lgr,rank_roe,rank_sgr,...,rank_baspread,rank_sue,rank_grltnoa,rank_std_turn,rank_depr,rank_cinvest,rank_op,rank_agr,rank_ep,log_me
date,,,,,,,,,,,,,,,,,,,,,
2013-07-31,0.106233,1.0,11.0,8.733313e+05,0.000000,-0.737624,-0.156328,0.000000,0.000000,0.000000,...,-0.228856,0.000000,0.000000,-0.668317,0.000000,0.000000,0.000000,0.000000,0.000000,13.680070
2013-08-31,0.035066,1.0,11.0,9.501110e+05,0.000000,-0.694578,-0.357430,0.000000,0.000000,0.000000,...,-0.013203,0.000000,0.000000,-0.408898,0.000000,0.000000,0.000000,0.000000,0.000000,13.764334
2013-09-30,0.089279,1.0,11.0,9.834276e+05,0.000000,-0.698404,-0.508903,0.000000,-0.109434,0.000000,...,0.363164,-0.468880,0.000000,-0.401991,0.000000,0.000000,0.000000,0.000000,0.000000,13.798799
2013-10-31,0.074094,1.0,11.0,1.081776e+06,0.000000,-0.535950,-0.387013,0.000000,-0.115681,0.000000,...,-0.698288,-0.284507,0.000000,0.332380,0.000000,0.000000,0.000000,0.000000,0.000000,13.894115
2013-11-30,0.066257,1.0,11.0,1.159195e+06,0.000000,-0.522464,-0.402050,0.000000,-0.050129,0.000000,...,0.113014,-0.249838,0.000000,0.249859,0.000000,0.000000,0.000000,0.000000,0.000000,13.963237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-0.112676,3.0,11.0,2.345556e+04,-0.444488,0.984442,0.712110,0.245117,-0.536906,0.011786,...,0.739678,0.741955,-0.609195,0.973928,0.971420,0.962517,-0.660321,-0.597799,-0.793579,10.062863
2022-09-30,-0.142857,3.0,11.0,2.081268e+04,-0.455121,0.984395,0.697556,0.237414,-0.534483,0.018765,...,-0.042951,0.736685,-0.612265,0.225547,0.971920,0.962426,-0.657766,-0.596149,-0.818037,9.943318
2022-10-31,-0.152778,3.0,11.0,1.783944e+04,-0.455197,0.984296,0.737638,0.231110,-0.534813,0.021938,...,-0.092155,0.736937,-0.609950,0.071356,0.971693,0.961642,-0.652667,-0.605868,-0.803481,9.789167


In [13]:
data_3d = data_3d.pivot_table(index=["date","permno"])
#data_3d

In [70]:
data_3d

exchcd        lag_me     log_me  rank_abr  rank_acc  \
date       permno                                                        
2013-05-31 87604      3.0  1.855514e+03   7.525917  1.000000  1.000000   
           90640      3.0  1.416794e+04   9.558737 -1.000000 -1.000000   
2013-06-30 10025      3.0  4.672376e+05  13.054593  0.274194  0.000000   
           10182      3.0  1.905212e+06  14.460104 -0.879032  0.000000   
           10259      3.0  1.567256e+05  11.962251  0.919355  0.000000   
...                   ...           ...        ...       ...       ...   
2022-12-31 93423      1.0  2.003252e+06  14.510282  0.639278 -0.084917   
           93426      1.0  5.091941e+05  13.140584  0.473416  0.222179   
           93429      3.0  1.345544e+07  16.414894  0.182905 -0.050019   
           93434      3.0  4.731153e+04  10.764509  0.929789 -0.640946   
           93436      3.0  6.148143e+08  20.236831 -0.010430  0.184180   

                   rank_agr  rank_alm  rank_ato  rank_baspread  rank_beta  \
date       permno                                                           
2013-05-31 87604   1.000000 -1.000000 -1.000000      -1.000000  -1.000000   
           90640  -1.000000  1.000000  1.000000       1.000000   1.000000   
2013-06-30 10025   0.000000  0.254098  0.000000      -0.405622  -0.469880   
           10182   0.000000  0.918033  0.000000      -0.839357  -0.566265   
           10259   0.000000  0.622951  0.000000       0.614458   0.012048   
...                     ...       ...       ...            ...        ...   
2022-12-31 93423  -0.543784 -0.306801 -0.189940       0.003798   0.736497   
           93426   0.005946 -0.054351  0.105228       0.056470  -0.191822   
           93429  -0.243784 -0.944758 -0.234282      -0.706255  -0.699142   
           93434  -0.644865  0.615088 -0.297816      -0.304634   0.669359   
           93436   0.587568 -0.792694  0.598279       0.318815   0.603231   

                   ...  rank_seas1a  rank_sgr   rank_sp  rank_std_dolvol  \
date       permno  ...                                                     
2013-05-31 87604   ...     1.000000 -1.000000  1.000000         1.000000   
           90640   ...    -1.000000  1.000000 -1.000000        -1.000000   
2013-06-30 10025   ...     0.000000  0.000000 -0.132530         0.028112   
           10182   ...     0.000000  0.000000  0.983936         0.180723   
           10259   ...     0.000000  0.000000  0.437751         0.389558   
...                ...          ...       ...       ...              ...   
2022-12-31 93423   ...     0.869702  0.714201  0.242233        -0.268551   
           93426   ...     0.538462  0.165228  0.176794         0.165068   
           93429   ...    -0.021978  0.884489 -0.546329        -0.376577   
           93434   ...    -0.425432 -0.851146  0.558427         0.592125   
           93436   ...    -0.463632  0.596904 -0.740445        -1.000000   

                   rank_std_turn  rank_sue  rank_turn  rank_zerotrade  \
date       permno                                                       
2013-05-31 87604        1.000000 -1.000000   1.000000       -1.000000   
           90640       -1.000000  1.000000  -1.000000        1.000000   
2013-06-30 10025       -0.421687  0.000000  -0.225806        0.044534   
           10182        0.068273  0.000000   0.137097        0.165992   
           10259        0.397590  0.000000  -0.209677       -0.319838   
...                          ...       ...        ...             ...   
2022-12-31 93423        0.779404 -0.299054   0.896071       -0.738318   
           93426       -0.575467  0.732861  -0.663878       -0.089164   
           93429       -0.032307  0.164894  -0.066160       -0.755999   
           93434       -0.832408  0.382979  -0.830672        0.867643   
           93436        0.434629  0.688534   0.752091       -0.885325   

                        ret  shrcd  
date       permno                   
2013-05-31 87604   0.024691   11.0  
    

In [41]:
"""
col_name_3d = list(data_3d)
col_name_3d.remove('date')
col_name_3d.remove('permno')
col_name_3d
"""

In [4]:
col_name_3d = data_3d.columns
col_3d = np.setdiff1d(col_name_3d, ['permno', 'date'])
print(col_3d)

['exchcd' 'lag_me' 'log_me' 'rank_abr' 'rank_acc' 'rank_agr' 'rank_alm'
 'rank_ato' 'rank_baspread' 'rank_beta' 'rank_bm' 'rank_cash'
 'rank_cashdebt' 'rank_cfp' 'rank_chcsho' 'rank_chpm' 'rank_chtx'
 'rank_cinvest' 'rank_depr' 'rank_dolvol' 'rank_ep' 'rank_gma'
 'rank_grltnoa' 'rank_ill' 'rank_lev' 'rank_lgr' 'rank_maxret' 'rank_me'
 'rank_mom12m' 'rank_mom1m' 'rank_mom36m' 'rank_mom60m' 'rank_mom6m'
 'rank_ni' 'rank_nincr' 'rank_noa' 'rank_op' 'rank_pctacc' 'rank_pm'
 'rank_pscore' 'rank_rd_sale' 'rank_rdm' 'rank_rna' 'rank_roa' 'rank_roe'
 'rank_rsup' 'rank_rvar_capm' 'rank_rvar_ff3' 'rank_rvar_mean'
 'rank_seas1a' 'rank_sgr' 'rank_sp' 'rank_std_dolvol' 'rank_std_turn'
 'rank_sue' 'rank_turn' 'rank_zerotrade' 'ret' 'shrcd']


In [58]:
data_3d.dtypes

permno                   int64
ret                    float64
exchcd                 float64
shrcd                  float64
date            datetime64[ns]
                     ...      
rank_cinvest           float64
rank_op                float64
rank_agr               float64
rank_ep                float64
log_me                 float64
Length: 61, dtype: object

In [95]:
basic_2_factors = Factor_models(data,20,freq='m') #2013年只有八年的数据

In [1]:
# 计算样本外R2，运行耗时较长
roos = basic_2_factors.cal_oos() # 计算不同模型样本外R2。self.cal_oos()中已经包含了self.predict_ret()的操作，先通过不同的模型预测收益率，再比较样本外真实收益率和预测收益率的差异
roos.to_csv('/home/mw/project/nn3_tuning_roos.csv')

In [ ]:
# 查看不同模型在不同月份的样本外预测ret
predict = basic_2_factors._preddf
predict.to_csv('/home/mw/project/AEencoder_predict.csv')